In [1]:
import pandas  as pd

dataset = pd.read_csv('../ozhegov_dataset.csv')
dataset = dataset[dataset['title'].str.len() > 0]
dataset = dataset[dataset['text'].str.len() > 0]
#для lenta - подсовываем даты
#dataset['title'] = dataset['date']
dataset = dataset.sample(n=1000)

dataset.astype({"text": str, "title": str})
dataset.info(show_counts=True)

dataset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 28934 to 16721
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1000 non-null   int64 
 1   title       1000 non-null   object
 2   text        1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 31.2+ KB


,Unnamed: 0,title,text
28934,28934,УТОЛЩЕНИЕ,Утолщенное место на чем-н. У. ствола. У. сосуда.
30193,30193,ЧЕРНОСОТЕНЕЦ,В России в нач. 20 в.: член шовинистической ор...
14378,14378,НЕПРЕОБОРИМЫЙ,"Такой, что невозможно побороть. Непреоборимая ..."
27420,27420,ТЕРМИЧЕСКИЙ,Относящийся к применению тепловой энергии в те...
27021,27021,СХЕМАТИЗИРОВАТЬ,Представить (-влять) в виде схемы (во 2 знач.)...


In [2]:
models = [
    "intfloat/multilingual-e5-large",
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/all-MiniLM-L6-v2"
    #"symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli",
    #"cointegrated/LaBSE-en-ru",
    #"sentence-transformers/LaBSE"
]

distances = [
    #"l2",
    #"ip",
    "cosine"
]

In [3]:
%pip install -U sentence-transformers ipywidgets chromadb chardet charset-normalizer

Note: you may need to restart the kernel to use updated packages.


Бывает ошибка с установкой, в самой ошибке есть решение

HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths

https://learn.microsoft.com/en-us/windows/win32/fileio/maximum-file-path-limitation?tabs=powershell#enable-long-paths-in-windows-10-version-1607-and-later

In [4]:
from chromadb.utils import embedding_functions
import chromadb
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

def create_collection(model_name, distance):
    
    chroma_client = chromadb.HttpClient(host="localhost", port=8000)

    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name)

    #text_collection = chroma_client.create_collection(name='text', embedding_function=sentence_transformer_ef)
    title_collection = chroma_client.create_collection(name="title", embedding_function=sentence_transformer_ef, metadata={"hnsw:space": distance})

    ids = list(map(str, dataset.index.values.tolist()))
    #text_collection.add(ids = ids, documents=dataset["text"].tolist())
    title_collection.add(ids = ids, documents=dataset["title"].tolist())

    return title_collection

def delete_collection():
    chroma_client.delete_collection("title")


def query_collection(collection, query, max_results, dataframe, model_name, distance):
    results = collection.query(query_texts=query, n_results=max_results, include=['distances']) 
    #print(results)
    df = pd.DataFrame({
                'id':results['ids'][0], 
                'score':list(map(float,results['distances'][0])),
                'query': query,
                'title': dataframe[dataframe.index.isin(list(map(int,results['ids'][0])))]['title'],
                'content': dataframe[dataframe.index.isin(list(map(int,results['ids'][0])))]['text'],
                'model_name': model_name,
                'distance': distance
                })
    
    # Забираем с минимальной дистанцией, значит он ближе и больше похож
    df = df[df.score == df.score.min()][:1]
    df['is_found'] = df.apply(lambda row: row.query == row.title, axis=1)
    
    return df


In [5]:
test_dataset = dataset.sample(n=100)
test_dataset.head()
test_results = pd.DataFrame()

In [6]:

for model in models:
    for distance in distances:
        print(f"{model} - {distance}")
        try:
            delete_collection()
        except Exception as ex:
            print(f"delete_collection error: {ex}")

        collection = create_collection(model, distance)

        for title in test_dataset["title"].tolist():
            test_results = test_results._append(query_collection(
            collection=collection,
            query=title,
            max_results=5,
            dataframe=dataset,
            model_name=model,
            distance=distance))

            print(f"{len(test_results)}")

        

test_results.to_csv("results_ozhegov2.csv")

intfloat/multilingual-e5-large - l2
delete_collection error: {"error":"ValueError('Collection title does not exist.')"}
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
intfloat/multilingual-e5-large - ip
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
intfloat/multilingual-e5-large - cosine
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228


In [9]:
test_results.head()

,id,score,query,title,content,model_name,distance,is_found
10363,10363,1.315708e-12,КОРНИШОНЫ,КОРНИШОНЫ,"Мелкие недозрелые огурцы, предназначенные для ...",intfloat/multilingual-e5-large,l2,True
8566,8566,7.252605e-13,ИММИГРАНТ,ИММИГРАНТ,"Человек, к-рый иммигрировал куда-н. II ж. имми...",intfloat/multilingual-e5-large,l2,True
12175,17352,1.157366e-12,ПЕНСИЯ,МЕНСТРУАЦИЯ,Ежемесячные выделения крови из матки женщины (...,intfloat/multilingual-e5-large,l2,False
18297,11029,7.939077e-13,КУТАТЬ,ПЛУТАТЬ,"Ходить не зная дороги, блуждать. П. по лесу.",intfloat/multilingual-e5-large,l2,False
14052,5394,1.371903e-12,ДЕКАДА,НЕДЕЛЯ,"Единица исчисления времени, равная семи дням, ...",intfloat/multilingual-e5-large,l2,False


In [7]:
finally_result = pd.DataFrame()
for model in models:
    for distance in distances:
        df = test_results.loc[test_results['model_name'].str.contains(model) == True]
        df = df.loc[df['distance'].str.contains(distance) == True]

        finally_result = finally_result._append(pd.DataFrame({
                'founded': [len(df[df['is_found'] == True])],
                'model_name': [model],
                'distance': [distance]
                }))
        
finally_result.head(15)


,founded,model_name,distance
0,24,intfloat/multilingual-e5-large,l2
0,24,intfloat/multilingual-e5-large,ip
0,24,intfloat/multilingual-e5-large,cosine
0,17,sentence-transformers/paraphrase-multilingual-...,l2
0,0,sentence-transformers/paraphrase-multilingual-...,ip
0,19,sentence-transformers/paraphrase-multilingual-...,cosine
0,23,symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli,l2
0,25,symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli,ip
0,21,symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli,cosine
0,14,cointegrated/LaBSE-en-ru,l2


In [8]:
finally_result.to_csv("finally_result_ozhegov2.csv")